In [ ]:
import pandas as pd
file1 = open('Urdu.txt', encoding="utf8")
rows = file1.readlines()
x = list()
for line in rows:
    x.append(line)

file2 = open('English.txt', encoding="utf8")
rows = file2.readlines()
y = list()
for line in rows:
    y.append(line)

In [ ]:
y_lines = pd.DataFrame(y)
x_lines = pd.DataFrame(x)

print(len(y_lines))
print(len(x_lines))

100000
100000


In [ ]:
from sklearn.model_selection import train_test_split
X_train_data, X_test, Y_train_data, Y_test_Data = train_test_split(x_lines, y_lines, test_size=0.15, random_state=1)
X_train_data, X_val, Y_train_data, Y_val_Data = train_test_split(X_train_data, Y_train_data, test_size=0.15, random_state=1) 
len(X_train_data)

72250

In [ ]:
pip install urduhack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 31.3 MB/s 
     |████████████████████████████████| 3.4 MB 61.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.5 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.6.0
    Uninstalling tensorflow-datasets-4.6.0:
      Successfully uninstalled tensorflow-datasets-4.6.0


In [ ]:
import urduhack
urduhack.download()

89078/89078 [==============================] - 0s 0us/step


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from urduhack.tokenization import word_tokenizer



SRC_LANGUAGE = 'ur'
TGT_LANGUAGE = 'en'

transform_token = {}
transform_vocab = {}

transform_token[SRC_LANGUAGE] = get_tokenizer(word_tokenizer)
transform_token[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# function to help yield list of tokens
def yield_tokens(data_iterator: Iterable, language: str) -> List[str]:
    index_of_lang = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for sample_data in data_iterator:
        yield transform_token[language](str(sample_data[index_of_lang[language]]))

# Defining special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

#before inserting into indices checking if tokens are in order
spec_symb = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # training iterator for data
    train_iterator = [X_train_data.values, Y_train_data.values]
    # creating vocabulary object for torch text 
    transform_vocab[ln] = build_vocab_from_iterator(yield_tokens(train_iterator, ln),
                                                    min_freq=1,
                                                    specials=spec_symb,
                                                    special_first=True)

# UNK_IDX = default index. returned when token not found.


# throws runtime error when querried token not found in vocabulary, if not set
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    transform_vocab[ln].set_default_index(UNK_IDX)

2/2 [==============================] - 1s 317ms/step


In [ ]:
tmpp = tuple(zip(X_train_data, Y_train_data))
tmpp[0]


(0, 0)

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class positional_encoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(positional_encoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        embed_pos = torch.zeros((maxlen, emb_size))
        embed_pos[:, 0::2] = torch.sin(pos * den)
        embed_pos[:, 1::2] = torch.cos(pos * den)
        embed_pos = embed_pos.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('embed_pos', embed_pos)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.embed_pos[:token_embedding.size(0), :])


class token_embedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(token_embedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Sequence to sequence learning Network
class seq_to_seq_transfer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(seq_to_seq_transfer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = token_embedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = token_embedding(tgt_vocab_size, emb_size)
        self.positional_encoding = positional_encoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                source_mask: Tensor,
                target_mask: Tensor,
                source_padding_mask: Tensor,
                target_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        source_embedding = self.positional_encoding(self.src_tok_emb(src))
        target_embedding = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(source_embedding, target_embedding, source_mask, target_mask, None,
                                source_padding_mask, target_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, source_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), source_mask)

    def decode(self, tgt: Tensor, memory: Tensor, target_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          target_mask)

In [ ]:
def square_subsqnt_mask_gen(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def make_mask(src, tgt):
    source_sequence_length = src.shape[0]
    target_sequence_length = tgt.shape[0]

    target_mask = square_subsqnt_mask_gen(target_sequence_length)
    source_mask = torch.zeros((source_sequence_length, source_sequence_length),device=DEVICE).type(torch.bool)

    source_padding_mask = (src == PAD_IDX).transpose(0, 1)
    target_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return source_mask, target_mask, source_padding_mask, target_padding_mask

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(transform_vocab[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(transform_vocab[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = seq_to_seq_transfer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for tmp in transformer.parameters():
    if tmp.dim() > 1:
        nn.init.xavier_uniform_(tmp)

transformer = transformer.to(DEVICE)

loss_function = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optim = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def seq_transf(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tens_transf(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))


text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = seq_transf(transform_token[ln], #Tokenization
                                               transform_vocab[ln], #Numericalization
                                               tens_transf) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    print(batch)
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](str(src_sample).rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](str(tgt_sample).rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
for x, y in tuple(zip(X_train_data.values, Y_train_data.values)):
    print(x)
    print(y)

Streaming output truncated to the last 5000 lines.
['(این ایل) مسٹر صدر ، جب بند دروازوں کے پیچھے ڈاسئیر تیار ہوجاتے ہیں تو ، میرے گروپ میں خطرے کی گھنٹیاں بجنے لگتی ہیں۔\n']
['(NL) Mr President, when dossiers are cooked up behind closed doors, alarm bells start ringing in my group.\n']
['اس نے ان کو رشوت کے طور پر بھی استعمال کیا تاکہ وہ عیسی کیمپ میں داخل ہونے میں مدد کریں۔\n']
['He also used them as bribes to help Avey to gain entrance to the Jewish camp.\n']
['آئین کے ان ٹکڑوں کو لوگوں کی مرضی کے خلاف ہونے پر مجبور نہیں کیا جاسکتا۔\n']
['These bits of the Constitution cannot be forced into being against the will of the people.\n']
['یکجہتی کی بدولت ہی اتحاد کا اصول ہمارے معاہدوں میں شامل ہوا۔\n']
['It was thanks to solidarity that the principle of cohesion came to be enshrined in our treaties.\n']
['وہ کونسا دل ہے جِس میں مکّاری نہیں اور وہ کونسی زبان ہے جو دو طرفہ نہیں ۔\n']
['Few hearts that are not double; few tongues that are not cloven.\n']
['ڈوب جانا  ۔ زوال پذیر  ہونا ۔\n']


In [ ]:
from torch.utils.data import DataLoader

def epoch_training(model, optim):
    model.train()
    losses = 0
    train_iterator = tuple(zip(X_train_data.values, Y_train_data.values))
    train_dataloader = DataLoader(train_iterator, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        source_mask, target_mask, source_padding_mask, target_padding_mask = make_mask(src, tgt_input)

        logits = model(src, tgt_input, source_mask, target_mask,source_padding_mask, target_padding_mask, source_padding_mask)

        optim.zero_grad()

        target_out = tgt[1:, :]
        loss = loss_function(logits.reshape(-1, logits.shape[-1]), target_out.reshape(-1))
        loss.backward()

        optim.step()
        losses += loss.item()

    return losses / len(train_dataloader)


def evaluate(model):
    model.eval()
    losses = 0

    value_iterator = tuple(zip(X_val.values, Y_val_Data.values))
    dataloader_for_val = DataLoader(value_iterator, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in dataloader_for_val:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        source_mask, target_mask, source_padding_mask, target_padding_mask = make_mask(src, tgt_input)

        logits = model(src, tgt_input, source_mask, target_mask,source_padding_mask, target_padding_mask, source_padding_mask)

        target_out = tgt[1:, :]
        loss = loss_function(logits.reshape(-1, logits.shape[-1]), target_out.reshape(-1))
        losses += loss.item()

    return losses / len(dataloader_for_val)

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 8
loss_best_val = 10000

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss_val = epoch_training(transformer, optim)
    end_time = timer()
    value_loss_val = evaluate(transformer)
    
    if loss_best_val > value_loss_val:
        loss_best_val = value_loss_val
        torch.save( transformer.state_dict() , "best_model_final.model" )
        
    print((f"Epoch: {epoch}, Train loss: {train_loss_val:.3f}, Val loss: {value_loss_val:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Streaming output truncated to the last 5000 lines.
      dtype=object)), (array(['دوستی کا  اظہار  ۔\n'], dtype=object), array(['A friendly gesture.\n'], dtype=object)), (array(['بہت ہُوا بھی تو پلکوں سے گدگدی کر دی\n'], dtype=object), array(['Even though it was raining, he tickled his eyelids\n'],
      dtype=object)), (array(['بیرون ملک یہ تہران میں آیت اللہ کے ساتھ ، کاسترو کے کیوبا کے ساتھ کاروبار کررہی ہے۔ وہ کمیونسٹ چین کو اسلحہ بیچنے کا حق مانگ رہا ہے۔\n'],
      dtype=object), array(["Abroad it is doing business with Castro's Cuba, with the Ayatollahs in Tehran; it is demanding the right to sell arms to Communist China.\n"],
      dtype=object)), (array(['دل کو دل سے راہ ہوتی ہے ۔\n'], dtype=object), array(['Confidence begets confidence.\n'], dtype=object)), (array(['ایک چھ سالہ اسکول والے بچے کی حیثیت سے ، ہمارے صدر بارک اوبامہ نے جکارتہ کی تنگ گندگی لینوں میں ننگے پاؤں کھیلے۔\n'],
      dtype=object), array(['As a six-year-old schoolboy, US President Barack Obama played baref

In [ ]:
def greedy_dec(model, src, source_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    memory = model.encode(src, source_mask)
    y_s = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        target_mask = (square_subsqnt_mask_gen(y_s.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(y_s, memory, target_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, n_word = torch.max(prob, dim=1)
        n_word = n_word.item()

        y_s = torch.cat([y_s,
                        torch.ones(1, 1).type_as(src.data).fill_(n_word)], dim=0)
        if n_word == EOS_IDX:
            break
    return y_s

In [ ]:
#actual function to transl input sentence into target language
def transl(model: torch.nn.Module, src_sentence: str):
    chceckpoint=torch.load('best_model_final.model')
    model.load_state_dict(chceckpoint)
    model.eval()
    
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    number_of_tokens = src.shape[0]
    source_mask = (torch.zeros(number_of_tokens, number_of_tokens)).type(torch.bool)
    target_tokens = greedy_dec(
        model,  src, source_mask, max_len=number_of_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(transform_vocab[TGT_LANGUAGE].lookup_tokens(list(target_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
print(transl(transformer, "بہت اچھا"))

NameError: ignored